In [1]:
import pandas as pd
import numpy as np

In [76]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [77]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


credits.head()

In [78]:
movies = movies.merge(credits,on='title')

In [79]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [80]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [81]:
movies = movies.dropna()

In [82]:
movies.duplicated().sum()

0

In [83]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [84]:
def convert(obj):
    lst = []
    for i in eval(obj):
        lst.append(i['name'])
    return lst

def fetch_director(obj):
    lst = []
    for i in eval(obj):
        if i['job'] == 'Director':
            lst.append(i['name'])
            break
    return lst

def characterLst(obj):
    lst = []
    for i in eval(obj)[:3]:
        lst.append(i['character'])
    return lst

In [85]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [86]:
movies['cast'] = movies['cast'].apply(characterLst)

In [87]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [88]:
movies['overview'] = movies['overview'].apply(lambda x:x.split("\W+"))

In [89]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Jake Sully, Neytiri, Dr. Grace Augustine]",[James Cameron]


In [93]:
movies['genres'] = movies['genres'].apply(lambda x:[i.lower().replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.lower().replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.lower().replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.lower().replace(" ","") for i in x])

In [94]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","[jakesully, neytiri, dr.graceaugustine]",[jamescameron]


In [100]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['overview']

In [101]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","[jakesully, neytiri, dr.graceaugustine]",[jamescameron],"[action, adventure, fantasy, sciencefiction, c..."


In [102]:
new_df = movies[['movie_id','title','tags']]

In [104]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_4632\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [106]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [115]:
import nltk

In [122]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [123]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [121]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_4632\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [124]:
from sklearn.feature_extraction.text import CountVectorizer

In [125]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [126]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [128]:
# cv.get_feature_names()

In [129]:
from sklearn.metrics.pairwise import cosine_similarity

In [132]:
similarity = cosine_similarity(vectors)

In [139]:
l = [1,5,6,0,2]
m = sorted(l,reverse=True)
[l.index(v) for v in m[:2]]

[2, 1]

In [158]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    dis = similarity[movie_index]
    max_dis = sorted(dis,reverse=True)
    recommend_movies_index = [dis.tolist().index(val) for val in max_dis[:5]]
    names = []
    for index in recommend_movies_index:
        names.append(new_df.iloc[index].title)
    return names

In [160]:
recommend("Avatar")

['Avatar',
 'Aliens vs Predator: Requiem',
 'Aliens',
 'Independence Day',
 'Titan A.E.']